In [1]:
import rdkit
import deepchem as dc

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'dgl'
Skipped loading some Jax models, missing a dependency. No module named 'haiku'


In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('/Users/libinathomas/repos/GBNN/Datasets/pdbbind_300.csv')
df = df.dropna()

In [4]:
df = df[:2]

In [5]:
df

Unnamed: 0 complex-name  gb-complex-etot  gb-complex-1-4-eel  \
0           0         10gs      -16145.9190          18478.8142   
1           1         1a1e       -9777.1684           7030.7012   

   gb-complex-eelec  gb-complex-egb  gb-complex-esurf  gb-protein-etot  \
0       -31052.1742      -3659.4630           86.9041      -16042.9095   
1       -13515.0036      -3349.3791           56.5130       -9499.3955   

   gb-protein-1-4-eel  gb-protein-eelect  ...  pb-protein-epb  \
0          18034.9833        -30493.1722  ...      -3650.6491   
1           7189.5223        -13366.9577  ...      -3333.0961   

   pb-protein-ecavity  pb-ligand-etot  pb-ligand-vdwaals  pb-ligand-eelec  \
0             87.9920       -565.1881           -10.1928        -416.8647   
1             56.2103       -173.5190           -11.4008         -96.6715   

   pb-ligand-epb  pb-ligand-ecavity       ddg  enthalpy-gb    entropy  
0      -141.8681             3.7375 -8.841927     -42.9160 -34.074073  
1       -69.9064             4.4598 -8.289306       3.2581  11.547406  

[2 rows x 35 columns]

In [6]:
complex_names_df = df['complex-name'].to_numpy()
PDBs = {}
from os import listdir
from os.path import isfile, join
mypath = '/Users/libinathomas/repos/GBNN/Datasets/pdbbind_complex'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
for f in onlyfiles:
    if f.split('.')[0] in complex_names_df:
        PDBs.update({f.split('.')[0] : rdkit.Chem.rdmolfiles.MolFromPDBFile(mypath + '/' + f)})

In [7]:
featurizer = dc.feat.ConvMolFeaturizer(per_atom_fragmentation=False)
TRAIN_SET = .8

In [8]:
X = []
X_ids = []
# one_add = 0 if len(PDBs.keys()) % 2 == 0 else 1
for k in PDBs.keys():
    X_ids.append(k)
    X.append(featurizer.featurize(PDBs[k]))
split_index = int(len(X) * TRAIN_SET)
X = [x[0] for x in X]
X_train_featurized = X[:split_index]
X_test_featurized = X[split_index:]
X_ids_train = X_ids[:split_index]
X_ids_test = X_ids[split_index:]

In [9]:
X_disk = []
y_disk = []
for i in range(len(X_ids)):
    X_disk.append(X[i])
    y_disk.append(df[df['complex-name'] == X_ids[i]]['ddg'].to_numpy()[0])
w_disk = np.ones([5,12])
X_add = []
for i in range(len(X_ids)):
    X_add.append(df[df['complex-name'] == X_ids[i]][[i for i in df.columns if ((('gb-' in i))
                                                          and ('-etot' not in i)) or ('-vdwaals' in i)]].to_numpy()[0])
train_dataset = dc.data.DiskDataset.from_numpy(X=X_disk, y=y_disk, w=w_disk, ids=X_ids)

In [10]:
x_add_train, x_add_test, y_train, y_test = [], [], [], []
for i in range(len(X_ids_train)):
    new_df = df[(df['complex-name'] == X_ids_train[i])]
    y_train.append(new_df['ddg'].to_numpy()[0])
    x_add_train.append(new_df[[i for i in df.columns if ((('gb-' in i))
                                                          and ('-etot' not in i)) or ('-vdwaals' in i)]].to_numpy()[0])
y_train = np.array(y_train)
    
for i in range(len(X_ids_test)):
    new_df = df[(df['complex-name'] == X_ids_train[i])]
    y_test.append(new_df['ddg'].to_numpy()[0])
    x_add_test.append(new_df[[i for i in df.columns if ((('gb-' in i))
                                                          and ('-etot' not in i)) or ('-vdwaals' in i)]].to_numpy()[0])
y_test = np.array(y_test)

In [11]:
from deepchem.metrics import to_one_hot
from deepchem.feat.mol_graphs import ConvMol

x_preprocessed_train, x_preprocessed_test = [], []

## for X train
multiConvMol = ConvMol.agglomerate_mols(X_train_featurized)
x_preprocessed_train = [multiConvMol.get_atom_features(), multiConvMol.deg_slice, np.array(multiConvMol.membership)]
for i in range(1, len(multiConvMol.get_deg_adjacency_lists())):
    x_preprocessed_train.append(multiConvMol.get_deg_adjacency_lists()[i])
x_preprocessed_train.append(np.array(x_add_train))

## for X test
multiConvMol = ConvMol.agglomerate_mols(X_test_featurized)
x_preprocessed_test = [multiConvMol.get_atom_features(), multiConvMol.deg_slice, np.array(multiConvMol.membership)]
for i in range(1, len(multiConvMol.get_deg_adjacency_lists())):
    x_preprocessed_test.append(multiConvMol.get_deg_adjacency_lists()[i])
x_preprocessed_test.append(np.array(x_add_test))

In [12]:
## !!!!!!!! important
## !!!!!!!! important
## !!!!!!!! important
## !!!!!!!! important
del(batch_size)

NameError: name 'batch_size' is not defined

In [13]:
X_add = np.array(X_add)

In [14]:
X_add

array([[ 1.84788142e+04, -3.10521742e+04, -3.65946300e+03,
         8.69041000e+01,  1.80349833e+04, -3.04931722e+04,
        -3.67271260e+03,  8.79920000e+01,  4.43830900e+02,
        -4.16864700e+02, -1.46368300e+02,  3.73750000e+00,
        -4.14363380e+03, -4.07786980e+03, -1.01928000e+01],
       [ 7.03070120e+03, -1.35150036e+04, -3.34937910e+03,
         5.65130000e+01,  7.18952230e+03, -1.33669577e+04,
        -3.37817030e+03,  5.62103000e+01, -1.58821000e+02,
        -9.66715000e+01, -6.91305000e+01,  4.45980000e+00,
        -1.95158170e+03, -1.90104860e+03, -1.14008000e+01]])

In [15]:
from deepchem.models.layers import GraphConv, GraphPool, GraphGather
import tensorflow as tf
import tensorflow.keras.layers as layers
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Concatenate
from tensorflow.keras import initializers


class GBGraphConvModel(tf.keras.Model):

  def modify_graphgather(self, batch_size):
    self.readout.batch_size = batch_size
    self.batch_size = batch_size
  def __init__(self, batch_size):
    super(GBGraphConvModel, self).__init__()
    self.counter = 0
    self.input_shapes = None
    self.batch_size = batch_size
    self.gc1 = GraphConv(32, activation_fn=tf.nn.tanh)
    self.batch_norm1 = layers.BatchNormalization()
    self.gp1 = GraphPool()

    self.gc2 = GraphConv(32, activation_fn=tf.nn.tanh)
    self.batch_norm2 = layers.BatchNormalization()
    self.gp2 = GraphPool()

    self.dense1 = layers.Dense(64, activation=tf.nn.tanh)
    self.batch_norm3 = layers.BatchNormalization()
    self.readout = GraphGather(batch_size=self.batch_size, activation_fn=tf.nn.tanh)

    self.dense2 = layers.Dense(1)
    self.dense3 = layers.Dense(1, 
         kernel_initializer=initializers.Constant([.5, -1, -1, 1, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1]),
         bias_initializer=initializers.Zeros())

  def call(self, inputs):
#     inputs = inputs[0]
    x = inputs
    x_add = X_add
#     input_shapes = [[4822, 75], [11, 2], [4822], [1142, 1], [1635, 2], [2042, 3],
#                    [3, 4], [0, 5], [0, 6], [0, 7], [0, 8], [0, 9], [0, 10]]
#     for i in range(len(self.input_shapes)):
#         x.append(tf.reshape(inputs[i][inputs[i] != 1.123456], self.input_shapes[i]))
#     for i in range(1, len(self.input_shapes)):
#         x[i] = tf.cast(x[i], tf.int32)
#     x_add = tf.reshape(inputs[13][inputs[13] != 1.123456], [self.batch_size, 15])
    gc1_output = self.gc1(x)
    batch_norm1_output = self.batch_norm1(gc1_output)
    gp1_output = self.gp1([batch_norm1_output] + x[1:])

    gc2_output = self.gc2([gp1_output] + x[1:])
    batch_norm2_output = self.batch_norm1(gc2_output)
    gp2_output = self.gp2([batch_norm2_output] + x[1:])

    dense1_output = self.dense1(gp2_output)
    batch_norm3_output = self.batch_norm3(dense1_output)
    readout_output = self.readout([batch_norm3_output] + x[1:])
    
    model_var = self.dense2(readout_output)
    binding_affinity = tf.concat([model_var, x_add], axis=1)
    ans = self.dense3(binding_affinity)
    ans = tf.reshape(ans, [1, -1])
    print(ans)
    return ans
model = GBGraphConvModel(split_index)
model.compile(loss='mse', optimizer='adam')

In [16]:
def loss_function(y, y_hat, w):
    return tf.keras.losses.mse(y_hat, y)
loss_func = lambda y, y_hat, w: tf.keras.losses.mse(y_hat, y)
model2 = dc.models.KerasModel(GBGraphConvModel(len(df)), loss = loss_function)

In [18]:
from deepchem.metrics import to_one_hot
from deepchem.feat.mol_graphs import ConvMol
import numpy as np
batch_size = len(df)
def data_generator(dataset, epochs=30):
    for ind, (X_b, y_b, w_b, ids_b) in enumerate(dataset.iterbatches(batch_size, epochs,
                                                                   deterministic=False, pad_batches=True)):
        multiConvMol = ConvMol.agglomerate_mols(X_b)
        inputs = [multiConvMol.get_atom_features(), multiConvMol.deg_slice, np.array(multiConvMol.membership)]
        for i in range(1, len(multiConvMol.get_deg_adjacency_lists())):
            inputs.append(multiConvMol.get_deg_adjacency_lists()[i])
#        inputs.append(X_add)
#         print(inputs[13])
        labels = y_b
        weights = [w_b]
        yield (inputs, labels, weights)

In [19]:
# model.input_shapes = [i.shape for i in x_preprocessed_train]
history = model.fit([x_preprocessed_train], y_train.reshape([1, -1]), epochs=1)

ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 3295, 11, 3295, 991, 1175, 1129, 0, 0, 0, 0, 0, 0, 0, 1
'y' sizes: 1


In [20]:
his = model2.fit_generator(data_generator(train_dataset, 1))

ImportError: `keras.optimizers.legacy` is not supported in Keras 3. When using `tf.keras`, to continue using a `tf.keras.optimizers.legacy` optimizer, you can install the `tf_keras` package (Keras 2) and set the environment variable `TF_USE_LEGACY_KERAS=True` to configure TensorFlow to use `tf_keras` when accessing `tf.keras`.

In [21]:
model.input_shapes = [i.shape for i in x_preprocessed_test]
model.modify_graphgather(len(X) - split_index)

In [22]:
model.evaluate(x_test, y_test.reshape([1, -1]))

NameError: name 'x_test' is not defined

In [23]:
np.sqrt(38.70)

6.220932405998316

In [26]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])

NameError: name 'history' is not defined

In [25]:
history = model.fit([x_preprocessed_train], y_train.reshape([1, -1]), epochs=1)

ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 3295, 11, 3295, 991, 1175, 1129, 0, 0, 0, 0, 0, 0, 0, 1
'y' sizes: 1


In [27]:

df.columns

Index(['Unnamed: 0', 'complex-name', 'gb-complex-etot', 'gb-complex-1-4-eel',
       'gb-complex-eelec', 'gb-complex-egb', 'gb-complex-esurf',
       'gb-protein-etot', 'gb-protein-1-4-eel', 'gb-protein-eelect',
       'gb-protein-egb', 'gb-protein-esurf', 'gb-ligand-etot',
       'gb-ligand-1-4-eel', 'gb-ligand-eelec', 'gb-ligand-egb',
       'gb-ligand-esurf', 'pb-complex-etot', 'pb-complex-vdwaals',
       'pb-complex-eelect', 'pb-complex-epb', 'pb-complex-ecavity',
       'pb-protein-etot', 'pb-protein-vdwaals', 'pb-protein-eelec',
       'pb-protein-epb', 'pb-protein-ecavity', 'pb-ligand-etot',
       'pb-ligand-vdwaals', 'pb-ligand-eelec', 'pb-ligand-epb',
       'pb-ligand-ecavity', 'ddg', 'enthalpy-gb', 'entropy'],
      dtype='object')

In [28]:
np.sqrt(np.mean((df['EX _H_(kcal/mol)'].to_numpy() - df['gb_delta_H'].to_numpy())**2))

KeyError: 'EX _H_(kcal/mol)'